In [1]:
from pprint import pprint
from data_algebra.data_ops import *
import data_algebra.env
import data_algebra.yaml
import data_algebra.PostgreSQL

db_model = data_algebra.PostgreSQL.PostgreSQLModel()

_, _1, _2, _get = [None, None, None, lambda x: x] # don't look unbound
data_algebra.env.push_onto_namespace_stack(locals())

t1 = TableDescription('t1', ['x', 'y'])

In [2]:
print(t1)

Table(t1; x, y)


In [3]:
print(t1.pretty_sql(db_model))


SELECT "x",
       "y"
FROM "t1"


In [4]:
ops = t1 >> Extend({'v':'x + 1'})

In [5]:
print(ops)

Table(t1; x, y) >>
   Extend({'v': (x + 1)})


In [6]:
print(ops.pretty_sql(db_model))

SELECT "x",
       "y",
       (x + 1) AS "v"
FROM
  (SELECT "x",
          "y"
   FROM "t1") "SQ_0"


In [7]:
t1b = TableDescription('t1', ['x', 'y'])
t2 = TableDescription('t2', ['x', 'z'])

ops = ( 
        t1 >> 
            Extend({'x':'x + 1'}) >> 
            NaturalJoin(
                b=t1b,
                by=['x', 'y']) >> 
            NaturalJoin(
                b=(t2 >> Extend({'f':'x + 1'})),
                by='x') 
)
print(ops)

Table(t1; x, y) >>
   Extend({'x': (x + 1)}) >>
   NaturalJoin(b=(
      Table(t1; x, y)),
      by=['x', 'y'], jointype=INNER) >>
   NaturalJoin(b=(
      Table(t2; x, z) >>
         Extend({'f': (x + 1)})),
      by=['x'], jointype=INNER)


In [8]:
print(ops.pretty_sql(db_model))

SELECT COALESE("LQ_8"."x", "RQ_9"."x") AS "x",
       COALESE("LQ_8"."y", "RQ_9"."y") AS "y"
FROM
  (SELECT COALESE("LQ_2"."x", "RQ_3"."x") AS "x",
          COALESE("LQ_2"."y", "RQ_3"."y") AS "y"
   FROM
     (SELECT "y",
             (x + 1) AS "x"
      FROM
        (SELECT "x",
                "y"
         FROM "t1") "SQ_0") "LQ_2"
   INNER JOIN
     (SELECT "y",
             (x + 1) AS "x"
      FROM
        (SELECT "x",
                "y"
         FROM "t1") "SQ_1") "RQ_3") "LQ_8"
INNER JOIN
  (SELECT COALESE("LQ_6"."x", "RQ_7"."x") AS "x",
          COALESE("LQ_6"."y", "RQ_7"."y") AS "y"
   FROM
     (SELECT "y",
             (x + 1) AS "x"
      FROM
        (SELECT "x",
                "y"
         FROM "t1") "SQ_4") "LQ_6"
   INNER JOIN
     (SELECT "y",
             (x + 1) AS "x"
      FROM
        (SELECT "x",
                "y"
         FROM "t1") "SQ_5") "RQ_7") "RQ_9"


In [9]:
p = ops.collect_representation()
pprint(p)

[OrderedDict([('op', 'TableDescription'),
              ('table_name', 't1'),
              ('qualifiers', {}),
              ('column_names', ['x', 'y']),
              ('key', 't1')]),
 OrderedDict([('op', 'Extend'), ('ops', {'x': '(x + 1)'})]),
 OrderedDict([('op', 'NaturalJoin'),
              ('by', ['x', 'y']),
              ('jointype', 'INNER'),
              ('b',
               [OrderedDict([('op', 'TableDescription'),
                             ('table_name', 't1'),
                             ('qualifiers', {}),
                             ('column_names', ['x', 'y']),
                             ('key', 't1')])])]),
 OrderedDict([('op', 'NaturalJoin'),
              ('by', ['x']),
              ('jointype', 'INNER'),
              ('b',
               [OrderedDict([('op', 'TableDescription'),
                             ('table_name', 't2'),
                             ('qualifiers', {}),
                             ('column_names', ['x', 'z']),
                   

In [10]:
import yaml
dmp = yaml.dump(p)
print(dmp)

- op: TableDescription
  table_name: t1
  qualifiers: {}
  column_names:
  - x
  - y
  key: t1
- op: Extend
  ops:
    x: (x + 1)
- op: NaturalJoin
  by:
  - x
  - y
  jointype: INNER
  b:
  - op: TableDescription
    table_name: t1
    qualifiers: {}
    column_names:
    - x
    - y
    key: t1
- op: NaturalJoin
  by:
  - x
  jointype: INNER
  b:
  - op: TableDescription
    table_name: t2
    qualifiers: {}
    column_names:
    - x
    - z
    key: t2
  - op: Extend
    ops:
      f: (x + 1)



In [11]:
ops_back = data_algebra.yaml.to_pipeline(yaml.safe_load(dmp))
print(ops_back)


Table(t1; x, y) >>
   Extend({'x': (x + 1)}) >>
   NaturalJoin(b=(
      Table(t1; x, y)),
      by=['x', 'y'], jointype=INNER) >>
   NaturalJoin(b=(
      Table(t2; x, z) >>
         Extend({'f': (x + 1)})),
      by=['x'], jointype=INNER)
